In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [3]:
df = pd.read_csv("02_Test_Data_Set/galaxy_users.csv")
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No


## Q1.

In [142]:
df_q1 = df[["OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies"]]
df_q1.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   OnlineSecurity    7032 non-null   object
 1   OnlineBackup      7032 non-null   object
 2   DeviceProtection  7032 non-null   object
 3   TechSupport       7032 non-null   object
 4   StreamingTV       7032 non-null   object
 5   StreamingMovies   7032 non-null   object
dtypes: object(6)
memory usage: 329.8+ KB


In [84]:
# (df_q1["OnlineBackup"] == "Yes") or (df_q1["OnlineBackup"] == "No")
print(df_q1["OnlineSecurity"].unique())
print(df_q1["OnlineBackup"].unique())
print(df_q1["DeviceProtection"].unique())
print(df_q1["TechSupport"].unique())
print(df_q1["StreamingTV"].unique())
print(df_q1["StreamingMovies"].unique())

['No' 'Yes' 'No internet service']
['Yes' 'No' 'No internet service']
['No' 'Yes' 'No internet service']
['No' 'Yes' 'No internet service']
['No' 'Yes' 'No internet service']
['No' 'Yes' 'No internet service']


In [85]:
df_q1 = df_q1.loc[~(df_q1["OnlineSecurity"] == 'No internet service'), ]
df_q1 = df_q1.loc[~(df_q1["OnlineBackup"] == 'No internet service'), ]
df_q1 = df_q1.loc[~(df_q1["DeviceProtection"] == 'No internet service'), ]
df_q1 = df_q1.loc[~(df_q1["TechSupport"] == 'No internet service'), ]
df_q1 = df_q1.loc[~(df_q1["StreamingTV"] == 'No internet service'), ]
df_q1 = df_q1.loc[~(df_q1["StreamingMovies"] == 'No internet service'), ]

df_q1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5512 entries, 0 to 7031
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   OnlineSecurity    5512 non-null   object
 1   OnlineBackup      5512 non-null   object
 2   DeviceProtection  5512 non-null   object
 3   TechSupport       5512 non-null   object
 4   StreamingTV       5512 non-null   object
 5   StreamingMovies   5512 non-null   object
dtypes: object(6)
memory usage: 301.4+ KB


In [86]:
dict(Yes = 1, No = 0)

{'Yes': 1, 'No': 0}

In [131]:
# df_q1["DeviceProtection"].unique()
df_q1 = df_q1.replace(dict(Yes = 1, No = 0))
df_q1.head(5)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,0,1,0,0,0,0
1,1,0,1,0,0,0
2,1,1,0,0,0,0
3,1,0,1,1,0,0
4,0,0,0,0,0,0


In [132]:
# 0, 1로 바꿔놓고 안 바꿔진걸 날리는 코드
df_q1 = df_q1.loc[df_q1["OnlineSecurity"].isin([0, 1]), ]
df_q1 = df_q1.loc[df_q1["OnlineBackup"].isin([0, 1]), ]
df_q1 = df_q1.loc[df_q1["DeviceProtection"].isin([0, 1]), ]
df_q1 = df_q1.loc[df_q1["TechSupport"].isin([0, 1]), ]
df_q1 = df_q1.loc[df_q1["StreamingTV"].isin([0, 1]), ]
df_q1 = df_q1.loc[df_q1["StreamingMovies"].isin([0, 1]), ]

df_q1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5512 entries, 0 to 7031
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   OnlineSecurity    5512 non-null   object
 1   OnlineBackup      5512 non-null   object
 2   DeviceProtection  5512 non-null   object
 3   TechSupport       5512 non-null   object
 4   StreamingTV       5512 non-null   object
 5   StreamingMovies   5512 non-null   object
dtypes: object(6)
memory usage: 301.4+ KB


In [147]:
# lambda 사용해서 한번에
df_q1["cnt"] = df_q1.apply(lambda x: sum(x == "Yes"), axis = 1)
ser_cnt = df_q1["cnt"].value_counts().sort_index()
print(round(ser_cnt[1] / ser_cnt[6], 1))
ser_cnt

3.4


/home/heesandv/miniconda3/envs/datascience/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0    2213
1     966
2    1033
3    1117
4     850
5     569
6     284
Name: cnt, dtype: int64

In [133]:
df_q1["cnt"] = df_q1.sum(axis = 1)

In [134]:
ser_cnt = df_q1["cnt"].value_counts().sort_index()
round(ser_cnt[1] / ser_cnt[6], 1)

3.4

## Q2.

In [91]:
from scipy.stats import pearsonr
df_q2 = df[["tenure", "MonthlyCharges", "TotalCharges"]]
df_q2.head(2)

,tenure,MonthlyCharges,TotalCharges
0,1,29.85,29.85
1,34,56.95,1889.50


In [96]:
df_q2["use_month"] = df_q2["TotalCharges"] // df_q2["MonthlyCharges"]
df_q2.reset_index()
df_q2.head(2)

/home/heesandv/miniconda3/envs/datascience/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tenure,MonthlyCharges,TotalCharges,use_month
0,1,29.85,29.85,1.0
1,34,56.95,1889.50,33.0


In [157]:
mat_corr = df_q2[["tenure", "MonthlyCharges", "use_month"]].corr(method="pearson")
mat_corr

,tenure,MonthlyCharges,use_month
tenure,1.000000,0.246862,0.998831
MonthlyCharges,0.246862,1.000000,0.246164
use_month,0.998831,0.246164,1.000000


In [158]:
mat_corr = mat_corr.reset_index()
mat_corr_melt = mat_corr.melt(id_vars="index")
mat_corr_melt

,index,variable,value
0,tenure,tenure,1.000000
1,MonthlyCharges,tenure,0.246862
2,use_month,tenure,0.998831
3,tenure,MonthlyCharges,0.246862
4,MonthlyCharges,MonthlyCharges,1.000000
5,use_month,MonthlyCharges,0.246164
6,tenure,use_month,0.998831
7,MonthlyCharges,use_month,0.246164
8,use_month,use_month,1.000000


In [161]:
mat_corr_melt = mat_corr_melt.loc[mat_corr_melt["index"] != mat_corr_melt["variable"], :]
mat_corr_melt

,index,variable,value
1,MonthlyCharges,tenure,0.246862
2,use_month,tenure,0.998831
3,tenure,MonthlyCharges,0.246862
5,use_month,MonthlyCharges,0.246164
6,tenure,use_month,0.998831
7,MonthlyCharges,use_month,0.246164


In [162]:
mat_corr_melt.loc[[mat_corr_melt["value"].abs().idxmax()], ]

,index,variable,value
2,use_month,tenure,0.998831


In [101]:
round(0.998831, 3)

0.999

## Q3.

In [136]:
df_q3 = df.copy()
df_q3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null   object 


In [137]:
df_q3["x1"] = df_q3["SeniorCitizen"]
df_q3["x2"] = df_q3["Partner"]
df_q3["x3"] = df_q3["Dependents"]
df_q3["x4"] = df_q3["tenure"]
df_q3["x5"] = df_q3["MonthlyCharges"]
df_q3["x6"] = df_q3["TotalCharges"]
df_q3["x7"] = df_q3["OnlineSecurity"]
df_q3["x8"] = df_q3["OnlineBackup"]
df_q3["x9"] = df_q3["DeviceProtection"]
df_q3["x10"] = df_q3["TechSupport"]
df_q3["x11"] = df_q3["StreamingMovies"]
df_q3["x11"] = df_q3["PaperlessBilling"]
df_q3["y"] = df_q3["Churn"]
df_q3 = df_q3.iloc[:, 21:]
df_q3.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,y
0,0,Yes,No,1,29.85,29.85,No,Yes,No,No,Yes,No
1,0,No,No,34,56.95,1889.50,Yes,No,Yes,No,No,No
2,0,No,No,2,53.85,108.15,Yes,Yes,No,No,Yes,Yes
3,0,No,No,45,42.30,1840.75,Yes,No,Yes,Yes,No,No
4,0,No,No,2,70.70,151.65,No,No,No,No,Yes,Yes


In [138]:
df_q3.apply(lambda x: x.unique())

x1                                                [0, 1]
x2                                             [Yes, No]
x3                                             [No, Yes]
x4     [1, 34, 2, 45, 8, 22, 10, 28, 62, 13, 16, 58, ...
x5     [29.85, 56.95, 53.85, 42.3, 70.7, 99.65, 89.1,...
x6     [29.85, 1889.5, 108.15, 1840.75, 151.65, 820.5...
x7                        [No, Yes, No internet service]
x8                        [Yes, No, No internet service]
x9                        [No, Yes, No internet service]
x10                       [No, Yes, No internet service]
x11                                            [Yes, No]
y                                              [No, Yes]
dtype: object

In [141]:
df_q3 = df_q3.replace({"Yes" : 1, "No" : 0, 'No internet service' : -1})
df_q3.head(5)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,y
0,0,1,0,1,29.85,29.85,0,1,0,0,1,0
1,0,0,0,34,56.95,1889.50,1,0,1,0,0,0
2,0,0,0,2,53.85,108.15,1,1,0,0,1,1
3,0,0,0,45,42.30,1840.75,1,0,1,1,0,0
4,0,0,0,2,70.70,151.65,0,0,0,0,1,1


In [148]:
df_q3_tr, df_q3_t = train_test_split(df_q3, train_size=0.7, random_state=123)

In [149]:
mmscaler = MinMaxScaler().fit(df_q3_tr)
df_q3_tr_nor = mmscaler.transform(df_q3_tr) # 결과물이 nd.array 가 됨
df_q3_t_nor = mmscaler.transform(df_q3_t)

# df_q3_tr_nor_df = pd.DataFrame(df_q3_tr_nor, columns = df_q3_tr_nor.columns)  <- 데이터프레임으로 변환해서도 진행 가능

In [168]:
df_q3_t_nor

array([[0. , 0. , 0. , ..., 0. , 1. , 0. ],
       [0. , 0. , 0. , ..., 1. , 1. , 0. ],
       [1. , 1. , 0. , ..., 0.5, 1. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 1. , 1. , ..., 0. , 1. , 0. ]])

In [172]:
model = LogisticRegression(random_state=123).fit(X = df_q3_tr_nor[:, :-1], y = df_q3_tr_nor[:, -1])
pred = model.predict(df_q3_t_nor[:, :-1])
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [173]:
round(f1_score(y_true=df_q3_t_nor[:, -1], y_pred=pred), 2)

0.55

#### 범주형 변수 한 번에 처리.

In [174]:
df2 = pd.read_csv("02_Test_Data_Set/galaxy_users.csv")
df2_sub = df2.loc[:, "Partner":"OnlineSecurity"]
df2_sub.head(2)

,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity
0,Yes,No,1,No,No phone service,DSL,No
1,No,No,34,Yes,No,DSL,Yes


In [178]:
df2_sub_obj = df2_sub.select_dtypes(exclude = "number")
df2_sub_obj.head(3)

,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity
0,Yes,No,No,No phone service,DSL,No
1,No,No,Yes,No,DSL,Yes
2,No,No,Yes,No,DSL,Yes


In [179]:
df2_sub_obj_melt = df2_sub_obj.melt()
df2_sub_obj_melt.head(3)

,variable,value
0,Partner,Yes
1,Partner,No
2,Partner,No


In [182]:
df2_sub_obj_melt["value"].unique()
# print(len(df2_sub_obj_melt["value"].unique()))

array(['Yes', 'No', 'No phone service', 'DSL', 'Fiber optic',
       'No internet service'], dtype=object)

In [183]:
ser_rep = pd.Series([1, 0, -1, -1, -1, -1],
                    index = df2_sub_obj_melt["value"].unique())
ser_rep

Yes                    1
No                     0
No phone service      -1
DSL                   -1
Fiber optic           -1
No internet service   -1
dtype: int64

In [184]:
df2_sub = df2_sub.replace(ser_rep)
df2_sub.head()

,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity
0,1,0,1,0,-1,-1,0
1,0,0,34,1,0,-1,1
2,0,0,2,1,0,-1,1
3,0,0,45,0,-1,-1,1
4,0,0,2,1,0,-1,0
